To specify a search space, define a model-building function it takes an hp argument, from which you can sample hyperparameter ranges, and it returns a compiled keras model.

# A keras model building function

In [1]:
from tensorflow import keras
from tensorflow.keras import layers

def build_model(hp):
    #sample hyperparameter values from the hp object
    #after sampling, these values (such as "units"
    #are just regular python constants)
    units = hp.Int(name="units", min_value=16, max_value=64, step=16)
    model = keras.Sequential([
        layers.Dense(units, activation="relu"),
        layers.Dense(units, activation="softmax")
    ])

    #different kin of hyperparametrs are available
    #Int, Float, Boolean, Choice
    optimizer = hp.Choice(name="optimizer", values=["rmsprop", "adam"])
    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    #the function returns a compiled model
    return model 


2024-01-29 14:53:11.027457: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-29 14:53:11.832512: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Using keras HyperModel

In [2]:
import keras_tuner as kt

class SimpleMLP(kt.HyperModel):
    def __init__(self, num_classes):
        self.num_classes = num_classes

    def build(self, hp):
        units = hp.Int(name="units", min_value=16, max_value=64, step=16)
        model = keras.Sequential([
            layers.Dense(units, activation="relu"),
            layers.Dense(self.num_classes, activation="softmax")
        ])
        optimizer = hp.Choice(name="optimizer", values=["rmsprop", "adam"])
        model.compile(
            optimizer=optimizer,
            loss="sparse_categorical_crossentropy",
            metrics=["accuracy"])
        return model

hypermodel = SimpleMLP(num_classes=10)
        

In [3]:
tuner = kt.BayesianOptimization(
    build_model,
    objective="val_accuracy",
    max_trials=100,
    executions_per_trial=2,
    directory="mnist_kt_test",
    overwrite=True,
)

2024-01-29 14:53:24.505428: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-29 14:53:24.650445: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [8]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 64, 'step': 16, 'sampling': 'linear'}
optimizer (Choice)
{'default': 'rmsprop', 'conditions': [], 'values': ['rmsprop', 'adam'], 'ordered': False}


In [4]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape((-1, 28 * 28)).astype("float32") / 255
x_test = x_test.reshape((-1, 28 * 28)).astype("float32") / 255
x_train_full = x_train[:]
y_train_full = y_train[:]
num_val_samples = 10000
x_train, x_val = x_train[:-num_val_samples], x_train[-num_val_samples:]
y_train, y_val = y_train[:-num_val_samples], y_train[-num_val_samples:]
callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=5),
]
tuner.search(
    x_train, y_train,
    batch_size=128,
    epochs=100,
    validation_data=(x_val, y_val),
    callbacks=callbacks,
    verbose=2,
)

Trial 100 Complete [00h 00m 38s]
val_accuracy: 0.9746000170707703

Best val_accuracy So Far: 0.9761500060558319
Total elapsed time: 01h 06m 52s


In [5]:
top_n = 4
best_hps = tuner.get_best_hyperparameters(top_n)

In [6]:
def get_best_epoch(hp):
    model = build_model(hp)
    callbacks=[
        keras.callbacks.EarlyStopping(
            monitor="val_loss", mode="min", patience=10)
    ]
    history = model.fit(
        x_train, y_train,
        validation_data=(x_val, y_val),
        epochs=100,
        batch_size=128,
        callbacks=callbacks)
    val_loss_per_epoch = history.history["val_loss"]
    best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch)) + 1
    print(f"Best epoch: {best_epoch}")
    return best_epoch

In [7]:
def get_best_trained_model(hp):
    best_epoch = get_best_epoch(hp)
    model = build_model(hp)
    model.fit(
        x_train_full, y_train_full,
        batch_size=128, epochs=int(best_epoch * 1.2))
    return model

best_models = []
for hp in best_hps:
    model = get_best_trained_model(hp)
    model.evaluate(x_test, y_test)
    best_models.append(model)

Epoch 1/100
391/391 [==============================] - 1s 3ms/step - loss: 0.5073 - accuracy: 0.8713 - val_loss: 0.2637 - val_accuracy: 0.9267
Epoch 2/100
391/391 [==============================] - 1s 3ms/step - loss: 0.2541 - accuracy: 0.9271 - val_loss: 0.2054 - val_accuracy: 0.9448
Epoch 3/100
391/391 [==============================] - 1s 3ms/step - loss: 0.2023 - accuracy: 0.9420 - val_loss: 0.1756 - val_accuracy: 0.9523
Epoch 4/100
391/391 [==============================] - 1s 3ms/step - loss: 0.1684 - accuracy: 0.9523 - val_loss: 0.1558 - val_accuracy: 0.9568
Epoch 5/100
391/391 [==============================] - 1s 3ms/step - loss: 0.1436 - accuracy: 0.9576 - val_loss: 0.1438 - val_accuracy: 0.9598
Epoch 6/100
391/391 [==============================] - 1s 3ms/step - loss: 0.1264 - accuracy: 0.9632 - val_loss: 0.1304 - val_accuracy: 0.9613
Epoch 7/100
391/391 [==============================] - 1s 3ms/step - loss: 0.1123 - accuracy: 0.9671 - val_loss: 0.1229 - val_accuracy: 0.9648